In [ ]:
import json
import os
from datetime import datetime

from langchain_core.prompts import PromptTemplate
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pathlib import Path

import whisperx_transcribe.utils as ut
from whisperx_transcribe import logger, start_time

In [ ]:
HISTORY = "../.history"
PROMPT_FILE = "../prompts/instructions_01.md"
SEG_FILE = "../assets/meeting_nvh_2025-07-25_diarized.json"
GROUP_SEG_FILE = f"../{SEG_FILE.split('.')[0]}_grouped.json"

In [ ]:
segments = ut.load_segments(segments_file=SEG_FILE)
grouped_segments = ut.group_speakers(segments=segments, speaker_names={})
with open(GROUP_SEG_FILE, "w") as f:
    json.dump(grouped_segments, f, indent=4)

with open(PROMPT_FILE, "r") as f:
    instructions = f.read()

In [ ]:
grouped_segments[10]

In [ ]:
ut.speaker_segment(grouped_segments[2])

In [ ]:
SPEAKERS = ut.get_all_speakers(segments=grouped_segments)
SPEAKERS

In [ ]:
model = "qwen2.5-coder:3b-instruct-q8_0"
llm = Ollama(model=model, request_timeout=1000)
embed_model = HuggingFaceEmbedding(
    "jinaai/jina-embeddings-v3",
    trust_remote_code=True,
    cache_folder="../model",
)
Settings.embed_model = embed_model
Settings.llm = llm

persist_dir = "storage"
Path(persist_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
QUERY_PROMPT = open("../prompts/user_final_01.md", "r").read()

In [ ]:
ut.speaker_segment(grouped_segments[0])

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext, SummaryIndex

index = SummaryIndex.from_documents(documents=[], show_progress=True)
# storage_context=StorageContext.from_defaults(persist_dir=persist_dir)
index.set_index_id(f"vector_index_{start_time}")
index.storage_context.persist(persist_dir)
len_segments = len(grouped_segments)
for idx, segment in enumerate(grouped_segments):
    doc = ut.speaker_segment(segment)
    index.insert(doc)
    # # every 20% of the segments, query the index
    # if (idx + 1) % (len_segments // 5) == 0:
    #     PROMPT = """**Task**
    #     1. Read the provided transcript
    #     2. Identify the important topics they addressed.

    #     **Formatting rules**

    #     - Use Markdown throughout.
    #     - Do **not** mention these instructions in your answer.

    #     **Input transcript structure**
    #     You have received the transcript for

    #     Review the transcript and perform the tasks above based on the provided instructions."""

    #     query_engine = index.as_query_engine(
    #         text_qa_template=PROMPT, response_mode="tree_summarize"
    #     )
    #     answer = query_engine.query(QUERY_PROMPT)
    #     print(answer)

In [ ]:
from llama_index.core.vector_stores import (
    FilterOperator,
    MetadataFilter,
    MetadataFilters,
    ExactMatchFilter,
)
from llama_index.core.prompts import RichPromptTemplate, SelectorPromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
def query_per_speaker(speaker: str):
    """Query the index for a specific speaker."""
    template_var_mappings = {
        "speaker": speaker,
    }
    base = RichPromptTemplate(
        template_str=open("../prompts/user_per_speaker_01.md", "r").read(),
        template_format="f-string",
        template_var_mappings=template_var_mappings,
    )

    filters = MetadataFilters(filters=[ExactMatchFilter(key="speaker", value=speaker)])
    qe = index.as_query_engine(
        # text_qa_template=base,
        # 'response_synthesizer:summary_template'
        summary_template=base,
        response_mode="tree_summarize",
        filters=filters,
    )
    # qe = RetrieverQueryEngine.from_args(
    #     retriever=retriever,
    #     llm=Settings.llm,
    #     summary_template=base,
    #     text_qa_template=base,
    #     response_mode="tree_summarize",
    # )
    prompts_dict = qe.get_prompts()
    print(prompts_dict)
    # return

    return qe.query(" ")  # query_str is not used.

In [ ]:
for speaker in SPEAKERS:
    response = query_per_speaker(speaker)
    print(f"Response for {speaker}:", response)
    print("\n" + "=" * 80 + "\n")
    break

In [ ]:
SUMMARY_PROMPT = """
You have received the transcript for a conversation. 

**Task**

1. Read the provided transcript and keep note of the speaker
2. Identify the key topics (1-2 points) they addressed.
3. Organize the topics into a clear, hierarchical structure.
4. After the turn by turn transcript has been reviewed for the full conversation, create an **Action‑Item table** with columns: `# | Action item | Responsible party / audience | Time reference`
5. Finish with a short **TL;DR / Core take‑aways** section (≤ 8 lines).

**Formatting rules**

- Use Markdown throughout.
- In the action table, keep **one action per row**.
- Do **not** mention these instructions in your answer.
"""

query_engine = index.as_query_engine(
    text_qa_template=SUMMARY_PROMPT, response_mode="tree_summarize"
)  # hierarchical
answer = query_engine.query("Give me a running bullet recap so far")
print(answer)